In [210]:
%matplotlib inline

import numpy as np
import random, csv

def read_file(filename):
    with open(filename, 'r') as file:
        lines = list(csv.reader(file))
    return lines

fires = read_file('forestfires.csv')

Сделаем следующие **преобразования данных**:
* уберем из рассмотрения поле day 
* добавим новую координату для свободного члена
* переведем нечисловые координаты в числовые
* перемешаем данные

In [211]:
for fire in fires:
    fire.pop(3) # remove day
    fire.append(1.0) # add free term

headings = fires[0]
fires = fires[1:]
for i in range(len(fires)):
    fires[i][2] = 1.0 if fires[i][2] in ['jun', 'jul', 'aug'] else 0.0
    fires[i] = [float(number) for number in fires[i]]

random.shuffle(fires)

Разобьем данные в соотношении 7:3. **Построим регрессионную модель**, применим модель ко второй части выборки и посчитаем
среднеквадратичную ошибку.

Далее применим к координате area преобразование  $f(x) = ln(c+x)$ для различных $c$. Посчитаем среднеквадратичную ошибку для преобразованных и для исходных значений.

In [212]:
def standart_deviation(function, sd_function):
    learn = fires[:(len(fires) * 7 // 10)]
    z_learn = np.matrix([fire[:-2] for fire in learn])
    x_learn = np.matrix([function(fire[-2]) for fire in learn]).transpose()
    theta_ls = np.linalg.inv(
        z_learn.transpose() * z_learn) * z_learn.transpose() * x_learn
    
    test = fires[(len(fires) * 7 // 10):]
    z_test = np.matrix([fire[:-2] for fire in test]) # [-2] - area
    x_test = np.matrix([function(x[-2]) for x in test]).transpose()
    result = z_test * theta_ls
    
    return np.mean(np.array(
            (sd_function(result) - sd_function(x_test)).transpose()[0]) ** 2)

print("x:", "%.2f" % standart_deviation(lambda x: x, lambda x: x), "\n")
for c in range(1, 100, 10):
    print("Преобразованных ln(" + str(c) + " + x):", standart_deviation(
            lambda x: np.log(c + x), lambda x: np.log(c + x)))
    print("Исходных к ln(" + str(c) + " + x):", standart_deviation(
            lambda x: np.log(c + x), lambda x: np.exp(x) - c))
    print("\n")

x: 742.19 

Преобразованных ln(1 + x): 0.411410026246
Исходных к ln(1 + x): 695.194307521


Преобразованных ln(11 + x): 0.00174292198762
Исходных к ln(11 + x): 659.114833889


Преобразованных ln(21 + x): 0.000324808484583
Исходных к ln(21 + x): 653.620146896


Преобразованных ln(31 + x): 0.000113268348522
Исходных к ln(31 + x): 658.479220878


Преобразованных ln(41 + x): 5.29398825432e-05
Исходных к ln(41 + x): 671.847355029


Преобразованных ln(51 + x): 2.94165718027e-05
Исходных к ln(51 + x): 693.456270096


Преобразованных ln(61 + x): 1.83325883485e-05
Исходных к ln(61 + x): 723.428374703


Преобразованных ln(71 + x): 1.23946529805e-05
Исходных к ln(71 + x): 762.009033389


Преобразованных ln(81 + x): 8.90272889171e-06
Исходных к ln(81 + x): 809.482309703


Преобразованных ln(91 + x): 6.69828724003e-06
Исходных к ln(91 + x): 866.139783194




Видим, что и в случае без преобразования, и в случае с преобразованием среднеквадратичная ошибка велика, но при малых
$c$ преобразование дает чуть лучшие предсказания.

Найдем $c$, при котором **среднеквадратичное отклонение минимально**.

In [213]:
sd_array = [standart_deviation(lambda x: np.log(c + x), 
                               lambda x: np.exp(x) - c)
            for c in np.arange(0.1, 100, 0.1)]
cmin = np.argmin(sd_array) / 10 - 1
print("c = " + str(cmin))
print("%.2f" % sd_array[np.argmin(sd_array)])

c = 19.6
653.62


Теперь попробуем разбить выборку на части 7:3 **разными способами** при полученном выше $c$.

In [214]:
for _ in range(100):
    random.shuffle(fires)
    print("%.2f" % standart_deviation(lambda x: np.log(cmin + x), 
                       lambda x: np.exp(x) - cmin))

8095.55
776.20
7955.47
1011.71
8058.63
680.42
7914.64
673.18
1054.81
485.54
11854.04
7788.14
680.14
516.30
444.45
496.28
803.75
458.81
1192.65
4278.92
1038.36
7924.90
934.93
8173.29
8353.64
945.45
557.23
8810.79
205.30
1188.97
911.08
604.42
4203.67
538.86
11612.58
1318.44
900.33
651.11
634.27
441.04
8403.74
11864.02
498.41
463.38
11622.25
1159.95
8007.24
170.17
847.84
4492.32
8534.11
183.00
3956.16
4276.56
403.35
935.53
8636.16
516.46
11645.65
11853.28
465.04
940.32
444.93
872.22
351.17
1145.90
4596.43
8072.43
971.69
790.32
1174.93
204.46
4689.93
634.77
4091.76
1271.88
8589.94
4133.30
309.00
4422.83
144.53
165.33
8064.01
1136.38
725.30
8201.78
1001.35
652.27
11171.96
4478.25
973.83
3917.28
11415.62
551.99
891.26
8646.80
11982.32
614.09
414.38
933.48


Видим, что в любом случае среднеквадратичное отклонение велико. К тому же, среднеквадратичное отклонение очень сильно зависит от способа разбиения. Поэтому линейная регрессия работает плохо.